In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np


In [ ]:

# Load Fashion-MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
print(f"Training data shape: {x_train.shape}")
print(f"Test data shape: {x_test.shape}")


Training data shape: (60000, 28, 28)
Test data shape: (10000, 28, 28)


In [ ]:

# Preprocessing
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(224, 224),
    layers.Rescaling(1./255)
])

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(lambda x, y: (resize_and_rescale(x), y))
train_ds = train_ds.shuffle(buffer_size=10000).batch(64)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(lambda x, y: (resize_and_rescale(x), y))
test_ds = test_ds.batch(64)


In [ ]:

# Build modified AlexNet
def build_alexnet():
    model = models.Sequential([
        layers.Conv2D(96, (11,11), strides=4, activation='relu', input_shape=(224,224,1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(3,3), strides=2),

        layers.Conv2D(256, (5,5), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(3,3), strides=2),

        layers.Conv2D(384, (3,3), padding='same', activation='relu'),
        layers.Conv2D(384, (3,3), padding='same', activation='relu'),
        layers.Conv2D(256, (3,3), padding='same', activation='relu'),
        layers.MaxPooling2D(pool_size=(3,3), strides=2),

        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    return model

model = build_alexnet()
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)         11712     
 batch_normalization (BatchN (None, 54, 54, 96)         384       
 max_pooling2d (MaxPooling2D (None, 26, 26, 96)         0         
 conv2d_1 (Conv2D)           (None, 26, 26, 256)        614656    
 batch_normalization_1 (Batc (None, 26, 26, 256)        1024      
 max_pooling2d_1 (MaxPooling (None, 12, 12, 256)        0         
 conv2d_2 (Conv2D)           (None, 12, 12, 384)        885120    
 conv2d_3 (Conv2D)           (None, 12, 12, 384)        1327488   
 conv2d_4 (Conv2D)           (None, 12, 12, 256)        884992    
 max_pooling2d_2 (MaxPooling (None, 5, 5, 256)          0         
 flatten (Flatten)           (None, 6400)               0         
 dense (Dense)               (None, 4096)               26218496  
 dropout (Dropout)           (None, 4096)  

In [ ]:

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:

# Train the model
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)



Epoch 1/10
938/938 [==============================] - 45s 48ms/step - loss: 1.3842 - accuracy: 0.5258 - val_loss: 0.8253 - val_accuracy: 0.7135
Epoch 2/10
938/938 [==============================] - 43s 46ms/step - loss: 0.6812 - accuracy: 0.7608 - val_loss: 0.6247 - val_accuracy: 0.7787
Epoch 3/10
938/938 [==============================] - 43s 46ms/step - loss: 0.5476 - accuracy: 0.8099 - val_loss: 0.5523 - val_accuracy: 0.8056
Epoch 4/10
938/938 [==============================] - 43s 46ms/step - loss: 0.4688 - accuracy: 0.8372 - val_loss: 0.4975 - val_accuracy: 0.8283
Epoch 5/10
938/938 [==============================] - 43s 46ms/step - loss: 0.4122 - accuracy: 0.8566 - val_loss: 0.4819 - val_accuracy: 0.8356
Epoch 6/10
938/938 [==============================] - 43s 46ms/step - loss: 0.3678 - accuracy: 0.8712 - val_loss: 0.4548 - val_accuracy: 0.8415
Epoch 7/10
938/938 [==============================] - 43s 46ms/step - loss: 0.3306 - accuracy: 0.8831 - val_loss: 0.4403 - val_accuracy

In [ ]:

# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)
print(f'\nTest accuracy: {test_acc:.4f}')



Test accuracy: 0.8604
